In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [ ]:
df = pd.read_csv(r"C:\Users\ashis\Dropbox\My PC (LAPTOP-UG4LH8KN)\Desktop\Akash\creditcard_marketing_dataset.csv")

In [ ]:
sns.histplot(df['ROI'], kde=True, color='blue')
plt.title('Distribution of ROI Across Campaigns')
plt.xlabel('ROI')
plt.ylabel('Frequency')
plt.show()

In [ ]:
sns.boxplot(x=df['CTR'], color='orange')
plt.title('Boxplot of Click-Through Rate (CTR)')
plt.xlabel('CTR')
plt.show()

Based on the boxplot of the Click-Through Rate (CTR) data:

- The median CTR appears to be between 5 to 6, as indicated by the center line in the box.
- The box spans a CTR range of around 4 to 7, suggesting the middle 50% of the data falls within this range.
- The whiskers extend to a minimum CTR around 3 and a maximum around 8, indicating the full range of observed CTR values.
- The lack of any outliers or data points beyond the whiskers suggests the CTR data is relatively compact and does not have any extreme outliers.

These insights could be useful for benchmarking performance and setting realistic targets for the credit card marketing analytics project. The centered median and compact distribution indicate the CTR metric is fairly stable, which may help in forecasting and optimizing marketing campaigns.

In [ ]:
# Distribution of Campaign Spend
fig = px.histogram(df, x='spend', color_discrete_sequence=['green'], 
    marginal="violin", log_x=True)
# Update layout to add title and axis labels
fig.update_layout(title='Distribution of Campaign Spend',xaxis_title='Spend',yaxis_title='Frequency',template='plotly_white')
fig.show()

1. The distribution appears right-skewed, with a large number of campaigns spending less than USD 100k, and fewer campaigns spending more. This suggests the majority of campaigns have relatively lower budgets.

2. There are spikes in spend around USD 100k and USD 1M, indicating these may be common or preferred spend levels for certain marketing initiatives or campaigns.

3. The overall range of spend is quite wide, from around USD 40k to over USD 1.5M, indicating a diverse set of campaign budgets and strategies being employed.

In [ ]:
sns.kdeplot(df['revenue_generated'], fill=True, color='purple')
plt.title('KDE Plot of Revenue Generated')
plt.xlabel('Revenue')
plt.show()

Based on the KDE (Kernel Density Estimation) plot of revenue generated :

1. The plot exhibits a highly right-skewed distribution, indicating that the majority of revenue generated is concentrated at the lower end of the scale, with a small number of campaigns driving very high revenue.

2. The prominent peak around the 2 revenue mark suggests there is a common, successful revenue-generating range that many campaigns tend to cluster around. This could represent a benchmark or target to aim for in optimizing marketing strategies.

3. The long tail extending to the right shows the potential for some campaigns to generate significantly higher revenue, up to around the 4-5 range. Understanding the drivers behind these high-performing outliers could provide valuable insights.

4. The rapid drop-off in density after the peak around 2 revenue suggests there may be challenges in consistently achieving substantially higher revenue, highlighting the importance of identifying and replicating the factors contributing to these top-performing campaigns.

These insights could inform about various aspects, such as setting realistic revenue targets, identifying high-potential strategies, and understanding the potential variance in outcomes across campaigns. Carefully analyzing this distribution can provide valuable guidance for optimizing the allocation of marketing resources and improving the overall performance of the credit card program.

In [ ]:
campaign_durations = (pd.to_datetime(df['end_date']) - pd.to_datetime(df['start_date'])).dt.days
sns.histplot(campaign_durations, bins=10, color='cyan')
plt.title('Campaign Duration Distribution')
plt.xlabel('Duration (Days)')
plt.ylabel('Number of Campaigns')
plt.show()

The most common campaign duration is around 20-40 days, with a large peak in the distribution around this range. This suggests that many campaigns are designed to run for about a month.
There is a significant drop-off in the number of campaigns after 50 days. Very few campaigns exceed 100 days in duration. This indicates that longer-running campaigns are less common.
The distribution is skewed, with a long tail of shorter campaign durations. This implies that while the majority of campaigns fall in the 20-60 day range, there are still a sizable number of much shorter campaigns as well.

In [ ]:
sns.histplot(df['impressions'], kde=True, color='darkblue', log_scale=True)
plt.title('Distribution of Campaign Impressions')
plt.xlabel('Impressions')
plt.ylabel('Frequency')
plt.show()

1. The distribution appears highly variable, with a few very high impression counts and many lower impression counts. This suggests a diverse set of campaigns and strategies being employed.

2. There are clear spikes in impressions around certain values, such as 1M and 2.5M impressions. These could represent common performance targets or thresholds for campaigns.

3. The long right tail of the distribution indicates some campaigns are able to generate very high impression volumes, potentially through effective targeting, ad placement, or other optimizations.

4. Understanding the factors that drive high impression campaigns versus lower impression campaigns could provide insights to improve the efficiency and effectiveness of future credit card marketing efforts.

In [ ]:
# Group data by start_date and calculate total spend
time_data = df.groupby('start_date')['spend'].sum().reset_index()
fig = px.line(time_data,x='start_date',y='spend',title='Spend Over Time',
    labels={'start_date': 'Start Date', 'spend': 'Total Spend'},
    color_discrete_sequence=['darkorange'])
# Layout
fig.update_layout(
    xaxis_title='Start Date',
    yaxis_title='Total Spend',
    xaxis_tickangle=45,
    template='plotly_white'
)
fig.show()

1. The spend shows significant fluctuations over time, with periodic spikes and dips. This suggests that there may be seasonality or other cyclical patterns in consumer spending behavior that could impact the effectiveness of marketing campaigns.

2. The overall trend appears to be increasing, with the most recent data points showing the highest total spend. This could indicate that the marketing efforts are having a positive impact on driving increased credit card usage and spending.

In [ ]:
from matplotlib.ticker import PercentFormatter
import textwrap

# Group and sort campaign revenue
campaign_revenue = (
    df.groupby('campaign_name')['revenue_generated']
    .sum()
    .sort_values(ascending=False)
)

# top 10 campaigns
top_campaigns = campaign_revenue.head(10)
cumulative_revenue = top_campaigns.cumsum() / top_campaigns.sum()

wrapped_labels = [
    '\n'.join(textwrap.wrap(name, width=10)) for name in top_campaigns.index
]

# Create the chart
fig, ax1 = plt.subplots(figsize=(10, 6))  # Increase figure size
ax1.bar(wrapped_labels, top_campaigns, color='skyblue')
ax2 = ax1.twinx()
ax2.plot(wrapped_labels, cumulative_revenue, color='orange', marker='o', linestyle='--')
ax2.axhline(0.8, color='red', linestyle='--', label='80% Threshold')
ax2.yaxis.set_major_formatter(PercentFormatter(1))

ax1.set_xlabel('Campaign Name')
ax1.set_ylabel('Revenue')
ax2.set_ylabel('Cumulative Percentage')
plt.title('Pareto Analysis of Campaign Revenue')

plt.xticks(rotation=45, ha='right') 
plt.legend(loc='upper left')
plt.tight_layout() 
plt.show()

1. The 80/20 rule: The 80/20 threshold line shows that 80% of the revenue is generated from around the top 20% of the campaigns. This suggests the company should focus its efforts on the most profitable campaigns.

2. Campaign performance: The chart shows a wide range of revenue generated by the different campaigns. "Holiday Rush Deals" is the highest revenue driver, while "Entreprener Excellence Rewards" is 10th. Understanding the drivers behind the top and bottom performers can inform future campaign planning.

3. Growth opportunities: Campaigns like "Digital Rewards" and "Super Saver Week" show strong revenue growth, indicating potential for expansion in those areas.

4. Incremental improvements: Even smaller campaigns like "Home Impro Big Ticket Savings" and "Fall Rewards Spectacular" contribute meaningful revenue. Identifying ways to enhance these can lead to incremental gains.

## Bivariate Analysis

In [ ]:
# Revenue vs Spend by Campaign
fig = px.scatter(df,x='spend',y='revenue_generated',color='campaign_name',size='conversions',hover_data=['campaign_name', 'ROI'],
                 title='Revenue vs Spend by Campaign')
fig.update_traces(marker=dict(opacity=0.7, line=dict(width=0.5, color='DarkSlateGrey')))
fig.show()

1. **Revenue Distribution**: The revenue generated varies significantly across different campaigns. The "Mid-Year Madness" campaign appears to be the most profitable, generating the highest revenue, while others like "Student Card Benefits" and "Streaming Service Offers" seem to have lower revenue contributions.

2. **Spending Efficiency**: The expenditure for each campaign is not detailed, but comparing the revenue to the implied spend (0.2M to 1M), it's clear that some campaigns are more cost-effective than others. For instance, if "Mid-Year Madness" has a higher spend but also significantly higher revenue, it might still be a worthwhile investment.

3. **Campaign Effectiveness**: Campaigns like "Luxury Card Exclusive Offer" and "Adventure Card Offers" show moderate revenue generation. These campaigns might need optimization to improve their ROI, possibly through better targeting or enhanced offers.

4. **Low-Performing Campaigns**: "Student Card Benefits" and "Streaming Service Offers" have the lowest revenue. This could indicate that the target audience for these campaigns is either not responding well or that the offers are not compelling enough. Re-evaluating the strategy for these segments could be beneficial.

5. **Seasonal Trends**: Campaigns like "Spring Break Bonanza" and "Holiday Rush Deals" might be seasonal, which could explain their performance. Understanding the timing and consumer behavior during these periods can help in planning future campaigns more effectively.

6. **Digital Focus**: The "Digital Deluxe Deals" campaign, which might be focused on digital channels, shows moderate success. This could suggest that digital marketing efforts are effective but may require further investment or refinement to maximize returns.

In [ ]:
# Conversions by Channel
channel_conversions = df.groupby('channel')['conversions'].sum().reset_index()
fig = px.bar(channel_conversions,x='channel',y='conversions',color='channel',
    title='Conversions by Channel',text='conversions')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(xaxis_title='Channel', yaxis_title='Total Conversions', showlegend=False)
fig.show()

1. **Channel Performance**: The total conversions vary across different channels. Facebook and Google Ads appear to be the most effective, with 22M and 25M conversions respectively. This suggests that these platforms are crucial for reaching and converting potential customers.

2. **Affiliate Marketing**: Affiliate marketing shows a significant contribution, with 22M conversions. This indicates that partnerships and affiliate networks are valuable for driving conversions and should be maintained or expanded.

3. **Social Media Impact**: Instagram and Twitter also contribute to conversions, with 18M and 29M respectively. This highlights the importance of social media presence and engagement in the marketing strategy.

4. **TV Advertising**: TV has 26M conversions, indicating that traditional media still plays a significant role in reaching a broad audience and driving conversions.

In [ ]:
# Channel Performance: Revenue vs ROI
channel_data = df.groupby('channel').agg({'revenue_generated': 'sum','ROI': 'mean','spend': 'sum'}).reset_index()
fig = px.scatter(channel_data,x='revenue_generated',y='ROI',size='spend',color='channel',
    title='Channel Performance: Revenue vs ROI',
    hover_data=['channel', 'spend'])
fig.show()

1. **Revenue and ROI Correlation**: The chart likely shows the relationship between revenue generated and the return on investment (ROI) for each channel. Channels with higher revenue and ROI are more efficient and should be prioritized.

2. **High-Performing Channels**: Channels like Facebook and Google Ads are expected to show strong performance in both revenue and ROI, indicating they are highly effective for credit card marketing campaigns.

3. **Affiliate Marketing**: If Affiliate Marketing shows a good balance of revenue and ROI, it suggests that partnerships and affiliate networks are valuable and should be maintained or expanded.

4. **Email Marketing**: Email might show moderate performance. This channel can be cost-effective and should be optimized for better engagement and conversion rates.

5. **Social Media Channels**: Instagram and Twitter may show varying levels of performance. Their effectiveness can depend on the target audience and the type of content shared. Continuous optimization of social media strategies is essential.

6. **TV Advertising**: TV might show significant revenue but potentially lower ROI due to higher costs. This indicates that while TV can reach a broad audience, it may require careful budgeting and targeting to improve ROI.

In [ ]:
# CTR vs Campaign Duration
fig = px.scatter(df,x='campaign_duration',y='CTR',size='impressions',color='campaign_name',hover_data=['start_date', 'end_date'],
    title='CTR vs Campaign Duration')
fig.show()

1. **CTR Performance**: The Click-Through Rate (CTR) varies across different campaigns. Campaigns like "Mid-Year Madness" and "Luxury Card Exclusive Offer" likely show higher CTRs, indicating more effective engagement with the target audience.

2. **Campaign Duration Impact**: The relationship between CTR and campaign duration suggests that longer campaigns do not necessarily result in higher CTRs. Some shorter campaigns might be more effective in capturing audience interest quickly.

3. **High-Performing Campaigns**: Campaigns with higher CTRs, such as "Mid-Year Madness," should be analyzed for best practices and strategies that can be replicated in other campaigns to improve overall performance.

4. **Low-Performing Campaigns**: Campaigns like "Student Card Benefits" and "Streaming Service Offers" may have lower CTRs. This could indicate a need for better targeting, more compelling offers, or improved creative content to enhance engagement.

5. **Seasonal Campaigns**: Seasonal campaigns like "Spring Break Bonanza" and "Holiday Rush Deals" might show varying CTRs based on the timing and relevance of the offers. Understanding seasonal trends can help in planning more effective campaigns.

In [ ]:
# Acquisition Cost vs. Revenue by Channel
df['CAC'] = df['spend'] / df['conversions']

fig = px.scatter(df,x='CAC',y='revenue_generated',size='conversions',color='channel',hover_data=['campaign_name', 'spend', 'impressions'],
    title='Acquisition Cost vs. Revenue by Channel',
    labels={'CAC': 'Customer Acquisition Cost', 'revenue_generated': 'Revenue Generated'})
fig.update_traces(marker=dict(opacity=0.7, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(title_font_size=18, legend_title='Channel')
fig.show()

1. **Revenue by Channel**: The revenue generated varies across different channels. Channels like Facebook, Google Ads, and Affiliate Marketing are likely to be top performers, contributing significantly to overall revenue.

2. **Customer Acquisition Cost (CAC)**: Channels with high revenue but low CAC are more cost-effective and should be prioritized.

3. **High-Performing Channels**: Facebook and Google Ads are expected to show strong revenue generation, indicating their effectiveness in reaching and converting potential customers.

4. **Affiliate Marketing**: If Affiliate Marketing shows substantial revenue, it suggests that partnerships and affiliate networks are valuable and should be maintained or expanded.

5. **Email Marketing**: Email might show moderate revenue. This channel can be cost-effective and should be optimized for better engagement and conversion rates.

6. **Social Media Channels**: Instagram and Twitter may show varying levels of revenue. Their effectiveness can depend on the target audience and the type of content shared. Continuous optimization of social media strategies is essential.

7. **TV Advertising**: TV might show significant revenue but potentially higher CAC due to the costs associated with traditional media. This indicates that while TV can reach a broad audience, it may require careful budgeting and targeting to improve cost-effectiveness.

In [ ]:
# Correlation Heatmap of Marketing Metrics
correlation_matrix = df[['spend', 'impressions', 'CTR', 'conversions', 'revenue_generated']].corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap of Marketing Metrics')
plt.show()

1. **Spend and Revenue**: There is a strong positive correlation (0.82) between spend and revenue_generated. This indicates that increased marketing spend is generally associated with higher revenue, suggesting that investment in marketing efforts is effective.

2. **Impressions and Conversions**: Impressions show a strong positive correlation (0.79) with conversions. This means that higher visibility of marketing campaigns leads to more conversions, emphasizing the importance of reach and frequency in marketing strategies.

3. **CTR and Conversions**: The correlation between CTR and conversions is moderate (0.26). This suggests that while higher click-through rates contribute to conversions, other factors also play a significant role in driving conversions.

4. **Spend and Impressions**: There is a positive correlation (0.62) between spend and impressions, indicating that higher spending leads to greater campaign visibility. This is expected as more budget allows for broader and more frequent ad placements.

5. **Revenue and Conversions**: The correlation between revenue_generated and conversions is moderate (0.64). This implies that while conversions are important, they are not the sole driver of revenue, and other factors such as customer lifetime value and average transaction size also contribute.

In [ ]:
# Distribution of Customer Lifetime Value (CLV) by Customer Segment and Gender
fig = px.violin(df, x='customer_segment', y='CLV', color='gender', box=True,
                title='Distribution of Customer Lifetime Value (CLV) by Customer Segment and Gender',
                labels={'CLV': 'Customer Lifetime Value', 'customer_segment': 'Customer Segment'})
fig.show()

1. **Gender Differences in CLV**: The image shows that there are noticeable differences in Customer Lifetime Value (CLV) between male and female customers across various customer segments. This suggests that gender-based targeting could be an effective strategy for maximizing CLV.

2. **Segment Performance**: The "Luxury Spenders" segment appears to have a higher CLV compared to "Tech Savvy" and "Budget Conscious" segments. This indicates that focusing on high-value customers in the Luxury Spenders segment could yield better returns.

3. **Budget Conscious Segment**: The CLV for the Budget Conscious segment is relatively lower, which might imply that customers in this segment are less profitable. However, they could still be valuable if acquired at a lower cost.

4. **Tech Savvy Segment**: The CLV for the Tech Savvy segment is moderate, suggesting that while they are not the highest spenders, they could still be a significant target group, especially if they are early adopters of new financial products.

5. **Marketing Strategies**: The data suggests that tailored marketing strategies for each segment and gender could enhance customer retention and CLV. For instance, personalized offers for Luxury Spenders and cost-effective solutions for Budget Conscious customers.

In [ ]:
# Revenue Over Time
time_series_data = df.groupby('last_purchase_date')['revenue_generated'].sum().reset_index()

fig = px.line(time_series_data, x='last_purchase_date', y='revenue_generated', 
              title='Revenue Over Time',
              labels={'last_purchase_date': 'Date', 'revenue_generated': 'Revenue Generated'})
fig.show()

In [ ]:
import plotly.express as px

fig = px.treemap(df, path=['campaign_name', 'channel_name'], values='revenue_generated',
                 color='revenue_generated', hover_data=['conversions'],
                 title='Treemap: Revenue by Campaign and Channel')
fig.show()

1. **Channel Performance**: The treemap indicates that different channels generate varying levels of revenue. TV appears to be a significant contributor, suggesting that traditional media still plays a crucial role in your marketing strategy.

2. **Digital Channels**: Twitter and Email also contribute notably to revenue, highlighting the importance of digital channels in reaching and engaging customers. This underscores the need to maintain a strong online presence.

3. **Campaign Effectiveness**: The revenue generated by different campaigns can help identify which strategies are most effective. For instance, campaigns run on TV and Twitter seem to be particularly successful, which could guide future budget allocations.

4. **Customer Engagement**: The data suggests that certain channels, like Twitter and Email, are effective in driving customer engagement and conversions. This could be leveraged for targeted marketing efforts, especially for tech-savvy or younger demographics.

5. **Cost Efficiency**: Analyzing the total cost versus revenue generated by each channel can help determine the cost efficiency of your campaigns. Channels with higher revenue relative to cost should be prioritized.

6. **Integrated Marketing**: The varying performance across channels indicates the potential benefits of an integrated marketing approach, combining traditional and digital channels to maximize reach and impact.

In [ ]:
fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = df['revenue_generated'].mean(),
    title = {'text': "Average Revenue Generated"},
    gauge = {'axis': {'range': [None, df['revenue_generated'].max()]},
             'steps' : [
                 {'range': [0, df['revenue_generated'].mean()], 'color': "lightgray"},
                 {'range': [df['revenue_generated'].mean(), df['revenue_generated'].max()], 'color': "gray"}],
             'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': df['revenue_generated'].mean()}}))

fig.show()

1. **Revenue Trends**: The average revenue generated varies significantly, with peaks at 7M and lows at 1M. This indicates fluctuations in revenue performance, which could be tied to specific campaigns, seasons, or market conditions.

2. **Performance Benchmarking**: The average revenue of 2.7M provides a benchmark for evaluating the success of your marketing efforts. Campaigns or strategies that consistently generate revenue above this average should be analyzed for best practices.

3. **High-Performing Periods**: The peak revenue of 7M suggests that there are periods or strategies that are particularly effective. Identifying the factors contributing to these high-performing periods can help replicate success in future campaigns.

4. **Low-Performing Periods**: The low revenue of 1M highlights areas that may require improvement. Investigating the causes of these dips can help in making necessary adjustments to avoid similar outcomes in the future.

5. **Customer Behavior**: The fluctuations in revenue could reflect changes in customer behavior or preferences. Analyzing these trends can provide insights into how to better align your offerings with customer needs and expectations.

In [ ]:
# Revenue Distribution by Channel
fig = px.pie(df, values='revenue_generated', names='channel_name',
             title='Pie Chart: Revenue Distribution by Channel',
             labels={'revenue_generated': 'Revenue Generated', 'channel_name': 'Channel Name'})
fig.show()

In [ ]:
# Calculate churn rate by customer segment
churn_data = df.groupby(['customer_segment', 'customer_churn']).size().unstack(fill_value=0)
churn_data['churn_rate'] = churn_data[1] / (churn_data[0] + churn_data[1])  # 1 = churned, 0 = not churned

# Merge with average CLV for each segment
clv_data = df.groupby('customer_segment')['CLV'].mean().reset_index()
churn_data = churn_data.merge(clv_data, on='customer_segment')

# Create a scatter plot
fig = px.scatter(churn_data, x='churn_rate', y='CLV', color='customer_segment', size='CLV',
                 title='Churn Rate vs. CLV by Customer Segment',
                 labels={'churn_rate': 'Churn Rate', 'CLV': 'Customer Lifetime Value'},
                 hover_name='customer_segment')
fig.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')))
fig.show()

1. **Customer Segment Analysis**: The churn rate varies across different customer segments. The "Budget Conscious" segment shows a higher churn rate compared to "Luxury Spenders" and "Tech Savvy" segments. This suggests that customers in the Budget Conscious segment are more likely to discontinue their services.

2. **Customer Lifetime Value (CLV)**: The CLV is relatively stable across the segments, with slight variations. This indicates that while churn rates differ, the overall value derived from customers in each segment remains consistent.

3. **Targeted Retention Strategies**: The higher churn rate in the Budget Conscious segment highlights the need for targeted retention strategies, such as personalized offers or loyalty programs, to reduce churn and retain these customers.

4. **Luxury Spenders**: The lower churn rate in the Luxury Spenders segment suggests that these customers are more loyal and less likely to switch. This segment should be prioritized for premium services and upselling opportunities.

5. **Tech Savvy Segment**: The Tech Savvy segment shows a moderate churn rate, indicating a balanced approach is needed. Engaging this segment with innovative digital solutions and tech-driven offers could help maintain their loyalty.

In [ ]:
# Churn Rate by Channel and Campaign
fig = px.bar(churn_by_channel_campaign, y='channel_name', x='churn_rate', color='campaign_name', 
             title='Churn Rate by Channel and Campaign',
             labels={'churn_rate': 'Churn Rate', 'channel_name': 'Channel Name', 'campaign_name': 'Campaign Name'},
             orientation='h')
fig.update_traces(width=0.5)

# Improve layout
fig.update_layout(
    yaxis_title='Channel Name',
    xaxis_title='Churn Rate',
    legend_title='Campaign Name',
    bargap=0.2,
    bargroupgap=0.1
)
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Calculate ROI and average CLV for each campaign
campaign_metrics = df.groupby('campaign_name').agg({
    'ROI': 'mean',
    'CLV': 'mean'
}).reset_index()

# Campaigns by ROI (descending)
campaign_metrics = campaign_metrics.sort_values(by='ROI', ascending=False)

# Create a dual-axis bar chart
fig = make_subplots(specs=[[{"secondary_y": True}]])

# ROI bars
fig.add_trace(
    go.Bar(x=campaign_metrics['campaign_name'], y=campaign_metrics['ROI'], name='ROI', marker_color='blue'),
    secondary_y=False
)

# CLV line
fig.add_trace(
    go.Scatter(x=campaign_metrics['campaign_name'], y=campaign_metrics['CLV'], name='CLV', mode='lines+markers', line=dict(color='red')),
    secondary_y=True
)

# layout
fig.update_layout(
    title='Campaign Ranking: ROI and CLV',
    xaxis_title='Campaign Name',
    yaxis_title='ROI',
    yaxis2_title='CLV',
    template='plotly_white',
    hovermode='x unified'
)
fig.show()

### **Insights:**

1. **Campaign Performance**: The image lists various campaigns, but the ranking based on ROI (Return on Investment) and CLV (Customer Lifetime Value) is not explicitly provided. However, the presence of campaigns like "Black Friday Bash," "Holiday Cashback Bonanza," and "Luxury Card Exclusive Offer" suggests that seasonal and premium campaigns tend to perform well due to their high appeal and customer engagement.

2. **High-Value Campaigns**: Campaigns targeting specific customer segments, such as "Young Professional Rewards," "Millennial Money Perks," and "Exclusive Gold Card Deals," are likely to resonate well with younger, high-spending demographics. These campaigns should be prioritized due to their potential for high CLV.

3. **Niche Campaigns**: Campaigns like "Petite Business Savings," "Mom & Pop Shop Rewards," and "Small Business Support" indicate a focus on niche markets. While these may not generate massive revenue, they can build long-term loyalty and brand advocacy.

4. **Seasonal Campaigns**: Seasonal campaigns like "Summer Travel Rewards," "Spring Break Bonanza," and "Holiday Shopping Extravaganza" are likely to perform well during specific times of the year. These campaigns should be optimized and scaled during their respective seasons.

5. **Low-Performing Campaigns**: Campaigns that lack clear targeting or differentiation, such as "Night Owl Offers" or "Weekly Wire Promotion," may not be generating significant ROI or CLV. These campaigns should be reviewed for effectiveness.

---

### **Recommendations:**

#### **Campaigns to Discontinue or Revamp:**
1. **Night Owl Offers**: Unless there is data showing strong engagement from late-night shoppers, this campaign may not be worth the investment. Consider discontinuing or revamping it with a more targeted approach.
   
2. **Weekly Wire Promotion**: If this campaign lacks a clear value proposition or fails to drive significant engagement, it should be discontinued or reimagined with a more compelling offer.

3. **Thromback Thursday Specials**: The name and concept may not resonate with customers. Consider rebranding or replacing it with a more relevant campaign.

4. **Virtual Shopper Bonuses**: If this campaign is not driving significant online sales or engagement, it may need to be revamped with better incentives or targeting.

5. **Common Equity Cashback**: This campaign sounds generic and may not stand out. Consider discontinuing it or tailoring it to a specific customer segment.

---

#### **Campaigns to Market with New Offers:**
1. **Luxury Card Exclusive Offer**: This campaign targets high-value customers and should be expanded with additional perks, such as concierge services or exclusive event access.

2. **Millennial Money Perks**: Millennials are a key demographic for credit card usage. Introduce new offers like cashback on streaming services, gym memberships, or eco-friendly purchases.

3. **Young Professional Rewards**: Tailor this campaign with offers that appeal to young professionals, such as discounts on co-working spaces, travel, or professional development courses.

4. **Holiday Cashback Bonanza**: Expand this campaign by offering tiered cashback rewards or bonus points for holiday spending.

5. **Streaming Subscriber Savings**: With the rise of digital entertainment, this campaign can be enhanced by partnering with popular streaming platforms for exclusive deals.

6. **Small Business Support**: Introduce new offers like cashback on business supplies or discounts on business services to strengthen loyalty among small business owners.

---

### **Strategic Focus:**
- **Data-Driven Optimization**: Use analytics to identify which campaigns are driving the highest ROI and CLV. Focus on scaling these campaigns while discontinuing or revamping underperforming ones.
- **Personalization**: Tailor campaigns to specific customer segments (e.g., millennials, luxury spenders, small businesses) to maximize engagement and retention.
- **Seasonal and Event-Based Campaigns**: Leverage seasonal opportunities (e.g., holidays, back-to-school, summer travel) to drive short-term spikes in revenue and long-term customer loyalty.

By focusing on high-performing campaigns and revamping or discontinuing underperforming ones, you can optimize your marketing efforts and maximize the ROI and CLV of your credit card offerings.

In [ ]:
import pandas as pd
import plotly.figure_factory as ff

# Calculate key metrics for each campaign
campaign_performance = df.groupby('campaign_name').agg({
    'ROI': 'mean',
    'CLV': 'mean',
    'revenue_generated': 'sum',
    'conversions': 'sum',
    'spend': 'sum'
}).reset_index()

# Define high-performing and low-performing campaigns
# High-performing: Top 25% by ROI
# Low-performing: Bottom 25% by ROI
high_performing = campaign_performance[campaign_performance['ROI'] >= campaign_performance['ROI'].quantile(0.75)]
low_performing = campaign_performance[campaign_performance['ROI'] <= campaign_performance['ROI'].quantile(0.25)]

# Add a column to distinguish high vs. low-performing campaigns
high_performing['Performance'] = 'High'
low_performing['Performance'] = 'Low'

# Combine into one DataFrame
performance_comparison = pd.concat([high_performing, low_performing])

# Create an interactive table
fig = ff.create_table(performance_comparison.sort_values(by='ROI', ascending=False))

# Add a title
fig.update_layout(
    title='Comparison Table: High-Performing vs. Low-Performing Campaigns',
    title_x=0.5
)

# Show the table
fig.show()

In [ ]:
import dash
import dash_table
import pandas as pd
from dash import html

# Calculate key metrics for each campaign
campaign_performance = df.groupby('campaign_name').agg({
    'ROI': 'mean',
    'CLV': 'mean',
    'revenue_generated': 'sum',
    'conversions': 'sum',
    'spend': 'sum'
}).reset_index()

# Define high-performing and low-performing campaigns
# High-performing: Top 25% by ROI
# Low-performing: Bottom 25% by ROI
high_performing = campaign_performance[campaign_performance['ROI'] >= campaign_performance['ROI'].quantile(0.75)]
low_performing = campaign_performance[campaign_performance['ROI'] <= campaign_performance['ROI'].quantile(0.20)]

# Add a column to distinguish high vs. low-performing campaigns
high_performing['Performance'] = 'High'
low_performing['Performance'] = 'Low'

# Combine into one DataFrame
performance_comparison = pd.concat([high_performing, low_performing])

# Initialize Dash app
app = dash.Dash(__name__)

# Create a layout with an interactive table
app.layout = html.Div([
    dash_table.DataTable(
        id='comparison-table',
        columns=[{"name": i, "id": i} for i in performance_comparison.columns],
        data=performance_comparison.to_dict('records'),
        sort_action='native',  # Enable sorting
        filter_action='native',  # Enable filtering
        style_table={'height': '400px', 'overflowY': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '10px'},
        style_header={'backgroundColor': 'lightblue', 'fontWeight': 'bold'},
        style_data_conditional=[
            {
                'if': {'filter_query': '{Performance} = "High"'},
                'backgroundColor': 'lightgreen',
                'color': 'black'
            },
            {
                'if': {'filter_query': '{Performance} = "Low"'},
                'backgroundColor': 'lightcoral',
                'color': 'black'
            }
        ]
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)